In [1]:
import neo4j 
import pandas as pd
from credentials import uri, user, pwd
from patent_neo4j.connection import Neo4jConnection
from patent_neo4j.analysis import get_direct_ancestor
from patent_neo4j.analysis import interpatent_inventor_combination
import itertools

Important Patents List

In [2]:
df = pd.read_csv("Data/important_patents.csv")
df.head(8)

,id,name
0,4136359,AppleMicrocomputer
1,4237224,MolecularChimeras
2,4371752,DigitalVoiceMailSystems
3,4399216,Co-transformationGeneCoding
4,4683195,PolymeraseChainReaction
5,5061620,StemCell
6,5108388,LaserSurgeryMethod
7,6285999,PageRank


Establishing Connection

### Inter-Patent Inventor Similarity

In [3]:
conn = Neo4jConnection(uri, user, pwd)
result = conn.query_citation_tree(root=df.id[0])

In [4]:
result.head()

,id,date,country,claims,kind,lineage,similarity
0,6226017,2001-05-01,US,48,A,[4136359],[0.057604003697633736]
1,9940907,2018-04-10,US,20,B2,"[6226017, 4136359]","[0.13514360785484314, 0.057604003697633736]"
2,7705842,2010-04-27,US,16,B2,"[6226017, 4136359]","[0.1497403383255005, 0.057604003697633736]"
3,9001133,2015-04-07,US,22,B2,"[7705842, 6226017, 4136359]","[0.2110498994588852, 0.1497403383255005, 0.057..."
4,9153179,2015-10-06,US,11,B2,"[7705842, 6226017, 4136359]","[0.3195275664329529, 0.1497403383255005, 0.057..."


In [5]:
# Get Direct Lineage
direct_lineage = get_direct_ancestor(result)

In [6]:
direct_lineage.head()

,id,lineage,similarity
0,6226017,4136359,0.057604
1,9940907,6226017,0.135144
2,7705842,6226017,0.149740
3,9001133,7705842,0.211050
4,9153179,7705842,0.319528


In [7]:
direct_lineage.lineage

0        4136359
1        6226017
2        6226017
3        7705842
4        7705842
          ...   
18850    7222306
18851    7222306
18852    7222306
18853    7222306
18854    7222306
Name: lineage, Length: 18855, dtype: object

In [8]:
inventor_tree = conn.query_inventor_tree(root=df.id[0])

In [11]:
interpatent = interpatent_inventor_combination(direct_ancestor=direct_lineage, inventor_tree=inventor_tree)

/home/jho/Jupyter/Patent/patent_neo4j/analysis.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direct_ancestor['combination'] = combination


In [14]:
for pair in interpatent['combination'][0]:
    print(pair)

('fl:a_ln:godfrey-5', 'fl:j_ln:goossen-4')
('fl:a_ln:godfrey-5', 'fl:s_ln:wozniak-4')
('fl:j_ln:goossen-4', 'fl:s_ln:wozniak-4')


In [15]:
from patent_neo4j.analysis import similarity_score, convert_np

In [ ]:
file_emb = "./node2vec/emb/" + df.name[0] + ".emb"
node_emb = convert_np(file_emb)